In [1]:
pip install scikit-learn
# pip show pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 간단한 데이터 전처리 (to DataFrame) 및 데이터베이스 삽입

In [6]:
import pymongo
import pandas as pd
import numpy as np

conn = pymongo.MongoClient()
db = conn.thread_data
major_col = db.major_data

# 원하는 변수
key_words = [
    'Spinbeam Temp',
    'Manifold #A Temp',
#     'Manifold #B Temp', # = Manifold #A Temp
    'GODET R/O 속도 A', # GR1_Speed
    'GODET R/O 온도 A', # GR1_Temp
    'GODET R/O 속도 B ', # GR2_Speed
    'GODET R/O 온도 B', # GR2_Temp
    '연신비', # DR
    '권취 R/O 속도', # FR_Speed
    '사인장강도', # Tenacity
    '사인장신도' # Elongation
]

# p : raw data가 존재하는 파일 경로
p = 'C:/Users/sunbum/Desktop/연구실/동계인턴/데이터/'
entire_data = pd.read_excel(
    p+'★기초데이터_방사분야_220922.xlsx', 
    header=[0, 1, 2])

# 필요 해더 추출 (2번째 해더)
col_list = list(map(lambda x : ''.join(x[1]), entire_data.columns))
entire_data.columns = col_list

# 필요 열 추출(projection according to 'key_words' list)
df1 = entire_data[key_words]

# 결측치 제거
df1 = df1.dropna(axis=0)
df1 = df1.reset_index(drop=True)

# 컬럼 이름 변경
df1.rename(columns={
    'Spinbeam Temp' : 'Spinbeam_Temp',
    'Manifold #A Temp' : 'Manifold_Temp',
    'GODET R/O 속도 A':'GR1_Speed',
    'GODET R/O 온도 A' : 'GR1_Temp',
    'GODET R/O 속도 B ' : 'GR2_Speed',
    'GODET R/O 온도 B' : 'GR2_Temp',
    '연신비' : 'DR',
    '권취 R/O 속도' : 'FR_Speed',
    '사인장강도' : 'Tenacity',
    '사인장신도' : 'Elongation'

}, inplace = True)

# MongoDB에 삽입하기 위한 JSON형식으로의 변환
key_value_list = []
keys = list(df1.columns)

for i in range(len(df1)):
    key_value = {}
    for j in range(len(keys)):
        if df1.loc[i, keys[j]].dtype == 'f8' or df1.loc[i, keys[j]].dtype == 'f4' or df1.loc[i, keys[j]].dtype == 'f2' :
            key_value[keys[j]] = float(df1.loc[i, keys[j]])
        elif df1.loc[i, keys[j]].dtype == 'i1' or df1.loc[i, keys[j]].dtype == 'i2' or df1.loc[i, keys[j]].dtype == 'i4' or df1.loc[i, keys[j]].dtype == 'i8':
            key_value[keys[j]] = int(df1.loc[i, keys[j]])
    key_value_list.append(key_value)

print('before insert : ' + str(major_col.count_documents({})))
major_col.insert_many(key_value_list)
print('after insert : ' + str(major_col.count_documents({})))

before insert : 815
after insert : 1635


In [162]:
# 데이터베이스 확인
major = major_col.find({}, {'_id' : 0})
for m in major:
    print(m)

{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 3500.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.154, 'FR_Speed': 4000, 'Tenacity': 2.75, 'Elongation': 59.2}
{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 3500.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.154, 'FR_Speed': 4000, 'Tenacity': 2.62, 'Elongation': 59.3}
{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 3500.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.154, 'FR_Speed': 4000, 'Tenacity': 2.59, 'Elongation': 55.4}
{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 3500.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.154, 'FR_Speed': 4000, 'Tenacity': 2.54, 'Elongation': 56.3}
{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 3000.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.3463333333333334, 'FR_Speed': 4000, 'Tenacity': 2.76, 'Elongation': 48.6}
{'Spinbeam_Temp': 256, 'Man

# Isolation Forest

In [132]:
from sklearn.ensemble import IsolationForest

# 이상치 탐지 모델 생성
isolation_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=1)

# Tenactiy 이상치 탐지 모델 학습
isolation_forest.fit(df1[['Tenacity']])

# Tenactiy 이상치 제거
df_Tena_out = df1[isolation_forest.predict(df1[['Tenacity']]) != -1]
df_Tena_out

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,Spinbeam_Temp,Manifold_Temp,GR1_Speed,GR1_Temp,GR2_Speed,GR2_Temp,DR,FR_Speed,Tenacity,Elongation
6,256,256,3000.0,90.0,4039.0,95.0,1.346333,4000,3.28,34.40
10,256,256,2500.0,90.0,4039.0,95.0,1.615600,4000,3.77,28.70
11,256,256,2500.0,90.0,4039.0,95.0,1.615600,4000,3.28,33.90
12,256,256,2000.0,90.0,4039.0,95.0,2.019500,4000,3.52,37.60
13,256,256,2000.0,90.0,4039.0,95.0,2.019500,4000,3.62,41.30
...,...,...,...,...,...,...,...,...,...,...
814,268,268,1490.0,90.0,4105.0,100.0,2.755034,4000,3.48,35.10
815,268,268,1490.0,90.0,4105.0,100.0,2.755034,4000,3.60,35.14
817,268,268,1500.0,90.0,4105.0,100.0,2.736667,4000,3.56,36.21
818,268,268,1500.0,90.0,4105.0,100.0,2.736667,4000,3.56,34.97


In [134]:
# Elongation 이상치 탐지 모델 학습
isolation_forest.fit(df1[['Elongation']])

# Elongation 이상치 제거
df_Elong_out = df1[isolation_forest.predict(df1[['Elongation']]) != -1]
df_Elong_out

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,Spinbeam_Temp,Manifold_Temp,GR1_Speed,GR1_Temp,GR2_Speed,GR2_Temp,DR,FR_Speed,Tenacity,Elongation
6,256,256,3000.0,90.0,4039.0,95.0,1.346333,4000,3.28,34.40
10,256,256,2500.0,90.0,4039.0,95.0,1.615600,4000,3.77,28.70
11,256,256,2500.0,90.0,4039.0,95.0,1.615600,4000,3.28,33.90
12,256,256,2000.0,90.0,4039.0,95.0,2.019500,4000,3.52,37.60
14,256,256,2000.0,90.0,4039.0,95.0,2.019500,4000,3.87,28.90
...,...,...,...,...,...,...,...,...,...,...
815,268,268,1490.0,90.0,4105.0,100.0,2.755034,4000,3.60,35.14
816,268,268,1490.0,90.0,4105.0,100.0,2.755034,4000,3.10,29.23
817,268,268,1500.0,90.0,4105.0,100.0,2.736667,4000,3.56,36.21
818,268,268,1500.0,90.0,4105.0,100.0,2.736667,4000,3.56,34.97


In [138]:
df_iTree = pd.concat([df_Elong_out,df_Tena_out])
df_iTree.drop_duplicates(keep='first', inplace=True, ignore_index=True)
# 정상 데이터 
df_iTree

# 이상치 데이터 
df_iTree_out = pd.merge(df1, df_iTree, how='outer', indicator = True)
df_iTree_out = df_iTree_out.query('_merge == "left_only"').drop(columns=['_merge'])
len(df_iTree_out)

16

In [148]:
from sklearn.ensemble import IsolationForest

keys = list(df1.columns)
keys.pop()
keys.pop()

def remove_outliers(group):
    model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.05, random_state=42)
    model.fit(group[['Tenacity', 'Elongation']])
    scores = model.decision_function(group[['Tenacity', 'Elongation']])
    threshold = scores.mean() + 2 * scores.std()
    outliers = group[scores < threshold]
    return group[scores >= threshold]


new_df = df1.groupby(keys).apply(remove_outlier)
new_df

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have va

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have va

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have va

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have va

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have va

Spinbeam_Temp  \
Spinbeam_Temp Manifold_Temp GR1_Speed GR1_Temp GR2_Speed GR2_Temp DR       FR_Speed                      
250           258           1240.0    90.0     4105.0    100.0    3.310484 4000     250            250   
                                                                                    251            250   
                                                                                    252            250   
                            1250.0    90.0     4105.0    100.0    3.284000 4000     253            250   
                                                                                    255            250   
...                                                                                                ...   
268           268           1480.0    90.0     4105.0    100.0    2.773649 4000     812            268   
                            1490.0    90.0     4105.0    100.0    2.755034 4000     814            268   
                                                                                    815            268   
                            1500.0    90.0     4105.0    100.0    2.736667 4000     817            268   
                                                                                    818            268   

                                                                                         Manifold_Temp  \
Spinbeam_Temp Manifold_Temp GR1_Speed GR1_Temp GR2_Speed GR2_Temp DR       FR_Speed                      
250           258           1240.0    90.0     4105.0    100.0    3.310484 4000     250            258   
                                                                                    251            258   
                                                                                    252            258   
                            1250.0    90.0     4105.0    100.0    3.284000 4000     253            258   
                                                                                    255            258   
...                                                                                                ...   
268           268           1480.0    90.0     4105.0    100.0    2.773649 4000     812            268   
                            1490.0    90.0     4105.0    100.0    2.755034 4000     814            268   
                                                                                    815            268   
                            1500.0    90.0     4105.0    100.0    2.736667 4000     817            268   
                                                                                    818            268   

                                                                                         GR1_Speed  \
Spinbeam_Temp Manifold_Temp GR1_Speed GR1_Temp GR2_Speed GR2_Temp DR       FR_Speed                  
250           258           1240.0    90.0     4105.0    100.0    3.310484 4000     250     1240.0   
                                                                                    251     1240.0   
                                                                                    252     1240.0   
                            1250.0    90.0     4105.0    100.0    3.284000 4000     253     1250.0   
                                                                                    255     1250.0   
...                                                                                            ...   
268           268           1480.0    90.0     4105.0    100.0    2.773649 4000     812     1480.0   
                            1490.0    90.0     4105.0    100.0    2.755034 4000     814     1490.0   
                                                                                    815     1490.0   
                            1500.0    90.0     4105.0    100.0    2.736667 4000     817     1500.0   
                                                                                    818     1500.0   

    

# IQR

In [79]:
import matplotlib.pyplot as plt

sample_df1 = df1.sample(n=4, replace=True) # 복원 추출

Q1 = sample_df1['Tenacity'].quantile(0.25)
Q3 = sample_df1['Tenacity'].quantile(0.75)

IQR = Q3-Q1
df_Tena = df1[(df1['Tenacity'] <= (Q3+1*IQR))]
df_Tena = df_Tena[(df_Tena['Tenacity'] >= (Q1-1*IQR))]

df_Tena_out = pd.merge(df1, df_Tena, how='outer', indicator = True)
df_Tena_out = df_Tena_out.query('_merge == "left_only"').drop(columns=['_merge'])
# 이상치 데이터
df_Tena_out

,Spinbeam_Temp,Manifold_Temp,GR1_Speed,GR1_Temp,GR2_Speed,GR2_Temp,DR,FR_Speed,Tenacity,Elongation
0,256,256,3500.0,90.0,4039.0,95.0,1.154000,4000,2.75,59.20
1,256,256,3500.0,90.0,4039.0,95.0,1.154000,4000,2.62,59.30
2,256,256,3500.0,90.0,4039.0,95.0,1.154000,4000,2.59,55.40
3,256,256,3500.0,90.0,4039.0,95.0,1.154000,4000,2.54,56.30
4,256,256,3000.0,90.0,4039.0,95.0,1.346333,4000,2.76,48.60
...,...,...,...,...,...,...,...,...,...,...
817,268,268,1490.0,90.0,4105.0,100.0,2.755034,4000,3.60,35.14
818,268,268,1490.0,90.0,4105.0,100.0,2.755034,4000,3.10,29.23
819,268,268,1500.0,90.0,4105.0,100.0,2.736667,4000,3.56,36.21
820,268,268,1500.0,90.0,4105.0,100.0,2.736667,4000,3.56,34.97


##### 정상 데이터
df_Tena

In [127]:
import pandas as pd
import numpy as np

keys = list(df1.columns)
keys.pop()
keys.pop()

def remove_outlier(group):
    Q1_Tena = group['Tenacity'].quantile(0.25)
    Q3_Tena = group['Tenacity'].quantile(0.75)
    
    Q1_Elong = group['Elongation'].quantile(0.25)
    Q3_Elong = group['Elongation'].quantile(0.75)
    
    IQR_Tena = Q3_Tena - Q1_Tena
    IQR_Elong = Q3_Elong - Q1_Elong
    
    lower_Tena = Q1_Tena - 1.57 * IQR_Tena
    upper_Tena = Q3_Tena + 1.57 * IQR_Tena
    lower_Elong = Q1_Elong - 1.57 * IQR_Elong
    upper_Elong = Q3_Elong + 1.57 * IQR_Elong
    return group[(group['Tenacity'] >= lower_Tena) & (group['Tenacity'] <= upper_Tena) & (group['Elongation'] >= lower_Elong) & (group['Elongation'] <= upper_Elong)]

IQR_df = df1.groupby(keys).apply(remove_outlier)
IQR_df

216

In [107]:
df_box_out = pd.concat([df_Elong_out,df_Tena_out])
df_box_out.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df_box_out

,Spinbeam_Temp,Manifold_Temp,GR1_Speed,GR1_Temp,GR2_Speed,GR2_Temp,DR,FR_Speed,Tenacity,Elongation
0,256,256,3000.0,90.0,4039.0,95.0,1.346333,4000,3.28,34.40
1,256,256,2500.0,90.0,4039.0,95.0,1.615600,4000,3.77,28.70
2,256,256,2500.0,90.0,4039.0,95.0,1.615600,4000,3.28,33.90
3,256,256,2000.0,90.0,4039.0,95.0,2.019500,4000,3.52,37.60
4,256,256,2000.0,90.0,4039.0,95.0,2.019500,4000,3.87,28.90
...,...,...,...,...,...,...,...,...,...,...
802,258,258,2000.0,90.0,4039.0,100.0,2.019500,4000,3.56,39.70
803,258,258,1256.0,90.0,4520.0,100.0,3.598726,4400,3.96,39.44
804,256,256,2000.0,90.0,4039.0,95.0,2.019500,4000,3.62,41.30
805,258,258,2000.0,90.0,4039.0,95.0,2.019500,4000,3.64,41.50


## 이외 trials

In [176]:
key_value_list = []
for i in range(len(df_iTree_out)):
    key_value = {}
    for j in range(len(keys)):
        if df_iTree_out.loc[i, keys[j]].dtype == 'f8' or df_iTree_out.loc[i, keys[j]].dtype == 'f4' or df_iTree_out.loc[i, keys[j]].dtype == 'f2' :
            key_value[keys[j]] = float(df_iTree_out.loc[i, keys[j]])
        elif df_iTree_out.loc[i, keys[j]].dtype == 'i1' or df_iTree_out.loc[i, keys[j]].dtype == 'i2' or df_iTree_out.loc[i, keys[j]].dtype == 'i4' or df_iTree_out.loc[i, keys[j]].dtype == 'i8':
            key_value[keys[j]] = int(df_iTree_out.loc[i, keys[j]])
    key_value_list.append(key_value)

print('before insert : ' + str(major_col.count_documents({})))
major_col.insert_many(key_value_list)
print('after insert : ' + str(major_col.count_documents({})))

before insert : 0
after insert : 815


In [9]:
major_col.delete_many({})

In [177]:
major = major_col.find({}, {'_id' : 0})
for m in major:
    print(m)

{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 3000.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.3463333333333334, 'FR_Speed': 4000, 'Tenacity': 2.76, 'Elongation': 48.6}
{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 3000.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.3463333333333334, 'FR_Speed': 4000, 'Tenacity': 3.28, 'Elongation': 34.4}
{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 2500.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.6156, 'FR_Speed': 4000, 'Tenacity': 3.11, 'Elongation': 43.8}
{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 2500.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.6156, 'FR_Speed': 4000, 'Tenacity': 3.16, 'Elongation': 44.7}
{'Spinbeam_Temp': 256, 'Manifold_Temp': 256, 'GR1_Speed': 2500.0, 'GR1_Temp': 90.0, 'GR2_Speed': 4039.0, 'GR2_Temp': 95.0, 'DR': 1.6156, 'FR_Speed': 4000, 'Tenacity': 3.77, 'Elongation': 28.7}
{'Spinbeam_

In [178]:
headers = []
key_values = []
data = major_col.find({}, {'_id' : 0})
for d in data:
    temp = []
    for k, v in d.items():
        temp.append(v)
        if k not in headers:
            headers.append(k)
    key_values.append(temp)
# headers
key_values

[[256, 256, 3000.0, 90.0, 4039.0, 95.0, 1.3463333333333334, 4000, 2.76, 48.6],
 [256, 256, 3000.0, 90.0, 4039.0, 95.0, 1.3463333333333334, 4000, 3.28, 34.4],
 [256, 256, 2500.0, 90.0, 4039.0, 95.0, 1.6156, 4000, 3.11, 43.8],
 [256, 256, 2500.0, 90.0, 4039.0, 95.0, 1.6156, 4000, 3.16, 44.7],
 [256, 256, 2500.0, 90.0, 4039.0, 95.0, 1.6156, 4000, 3.77, 28.7],
 [256, 256, 2500.0, 90.0, 4039.0, 95.0, 1.6156, 4000, 3.28, 33.9],
 [256, 256, 2000.0, 90.0, 4039.0, 95.0, 2.0195, 4000, 3.52, 37.6],
 [256, 256, 2000.0, 90.0, 4039.0, 95.0, 2.0195, 4000, 3.62, 41.3],
 [256, 256, 2000.0, 90.0, 4039.0, 95.0, 2.0195, 4000, 3.87, 28.9],
 [256, 256, 2000.0, 90.0, 4039.0, 95.0, 2.0195, 4000, 3.44, 34.8],
 [256, 256, 1500.0, 90.0, 4039.0, 95.0, 2.6926666666666668, 4000, 3.89, 29.5],
 [256, 256, 1500.0, 90.0, 4039.0, 95.0, 2.6926666666666668, 4000, 4.04, 32.4],
 [256, 256, 1500.0, 90.0, 4039.0, 95.0, 2.6926666666666668, 4000, 3.82, 28.5],
 [256, 256, 1500.0, 90.0, 4039.0, 95.0, 2.6926666666666668, 4000, 3.6

In [183]:
from datetime import datetime

today = datetime.now()

df = pd.DataFrame(data=key_values, columns=headers)

td = today.strftime('%Y%m%d')
# 인덱스 번호를 없애고 열 이름을 헤더로 지정
df.to_excel(excel_writer=p+'major_data_'+ td +'.xlsx', index=False, header=headers)